In [8]:
import os
import sys

# STOCKA_BASE_DIR = "/home/zzy/projects/stocka"
STOCKA_BASE_DIR = "/Users/zhaozhiyu/Projects/stocka"
sys.path.insert(0, STOCKA_BASE_DIR)

import pandas as pd

from datetime import date
from quant_framework.data.data_handler import DataHandler
from quant_framework.factor.alpha158 import Alpha158

In [9]:
data_handler = DataHandler(
    db_path=os.path.join(STOCKA_BASE_DIR, 'data/stock.db'),
)

In [10]:
for factor_id, factor_name, factor_category, factor_desc in Alpha158.DEFINITIONS:
    data_handler.register_factor(factor_id, factor_name, factor_category, factor_desc)

In [11]:
end_date = date(2026, 2, 5)
start_date = data_handler.get_previous_trading_date(end_date, 60)
start_date

datetime.date(2025, 11, 11)

In [12]:
from tqdm import tqdm

factor_df = None
# code_list = data_handler.get_all_codes()
code_list = ['000001', '000002', '600000']
pbar = tqdm(code_list)
alpha158 = Alpha158()
for code in pbar:
    pbar.set_postfix({"code": code})
    df = data_handler.get_range_data([code], start_date, end_date)
    new_factor_row = alpha158.calculate(df).iloc[[-1]]
    if factor_df is None:
        factor_df = new_factor_row
    else:
        # 使用 ignore_index=False 保留原始索引
        factor_df = pd.concat([factor_df, new_factor_row], ignore_index=False)

# 设置 code 列为索引，确保 save_factors 能正确获取股票代码
factor_df = factor_df.set_index('code')

100%|██████████| 3/3 [00:00<00:00, 18.39it/s, code=600000]


In [13]:
factor_df

,date,open,high,low,close,volume,amount,outstanding_share,turnover,KMID,...,VSUMN5,VSUMN10,VSUMN20,VSUMN30,VSUMN60,VSUMD5,VSUMD10,VSUMD20,VSUMD30,VSUMD60
code,,,,,,,,,,,,,,,,,,,,,
000001,2026-02-05,1602.25,1621.22,1599.33,1618.30,117759109.0,1.301134e+09,1.940560e+10,0.006068,0.009918,...,0.677496,0.44278,0.492652,0.463789,0.488868,-0.354991,0.114441,0.014696,0.072422,0.022264
000002,2026-02-05,822.05,832.19,818.66,825.43,214093420.0,1.043759e+09,9.716400e+09,0.022034,0.004095,...,0.754128,0.52902,0.520412,0.498831,0.488170,-0.508257,-0.058039,-0.040824,0.002338,0.023660
600000,2026-02-05,168.56,170.72,168.23,169.89,97223688.0,9.927789e+08,3.330584e+10,0.002919,0.007829,...,0.793122,0.58639,0.488276,0.481131,0.494406,-0.586243,-0.172779,0.023448,0.037737,0.011188


In [14]:
factor_names = [d[1] for d in Alpha158.DEFINITIONS]
data_handler.save_factors(factor_df[factor_names], "2026-02-05")

In [17]:
daily_factor = data_handler.get_factor_cross_section(date(2026, 2, 5))
daily_factor

factor_name,BETA10,BETA20,BETA30,BETA5,BETA60,CNTD10,CNTD20,CNTD30,CNTD5,CNTD60,...,VSUMP10,VSUMP20,VSUMP30,VSUMP5,VSUMP60,WVMA10,WVMA20,WVMA30,WVMA5,WVMA60
stock_code,,,,,,,,,,,,,,,,,,,,,
000001,0.000219,-0.002879,-0.002613,0.005681,-0.001226,-0.2,-0.35,-0.300000,0.2,-0.183333,...,0.557220,0.507348,0.536211,0.322504,0.511132,0.738294,0.822100,0.910144,0.677393,0.881267
000002,-0.001379,0.000125,0.000953,0.004508,-0.004557,-0.1,-0.10,-0.066667,-0.2,-0.300000,...,0.470980,0.479588,0.501169,0.245872,0.511830,1.088467,1.109369,1.232112,0.865491,1.611146
000006,-0.006478,-0.005421,-0.004720,0.004781,-0.004106,-0.2,-0.25,-0.033333,-0.2,0.016667,...,0.290916,0.467677,0.484221,0.243977,0.480954,0.874152,0.921501,0.924910,0.484834,0.967775
000007,-0.004616,-0.003432,-0.001940,0.007424,0.002996,0.1,0.05,-0.033333,0.4,0.083333,...,0.489841,0.316178,0.462607,0.197691,0.437744,0.746140,0.799195,1.141068,0.917348,1.671167
000008,-0.012778,-0.001915,0.002225,-0.007109,-0.000494,-0.2,-0.05,0.033333,-0.2,-0.050000,...,0.323338,0.419179,0.496053,0.242742,0.421842,1.461111,1.271822,1.877392,1.083104,1.850337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605580,-0.015814,-0.003111,-0.000113,-0.005809,-0.015855,0.0,0.20,-0.033333,0.2,-0.200000,...,0.246277,0.427344,0.433706,0.073016,0.462872,1.052772,1.252865,1.229509,0.691945,1.309945
605588,-0.006446,-0.000980,0.004272,-0.004332,0.002428,0.0,-0.10,0.066667,-0.2,0.033333,...,0.159552,0.308491,0.484248,0.000000,0.495472,0.857966,1.839921,1.814630,1.171473,1.967953
605589,-0.005286,0.000367,0.001532,-0.003622,0.002552,0.2,0.20,0.066667,0.2,0.066667,...,0.382607,0.434864,0.473590,0.153589,0.504263,0.539041,0.685833,1.075473,0.536382,1.163401


In [22]:
daily_stock = data_handler.get_daily_data(date(2026, 2, 5)).set_index('code')
daily_stock

,date,open,high,low,close,volume,amount,outstanding_share,turnover
code,,,,,,,,,
601599,2026-02-05,14.61,16.02,14.61,16.02,46393702.0,222835685.0,1.160542e+09,0.039976
600985,2026-02-05,64.26,64.26,62.25,63.30,38946182.0,489317462.0,2.693259e+09,0.014461
600609,2026-02-05,131.88,132.43,130.49,130.77,12058341.0,57167454.0,1.304559e+09,0.009243
002700,2026-02-05,45.85,45.97,45.23,45.36,6959400.0,51338401.0,2.634649e+08,0.026415
000066,2026-02-05,184.24,185.98,183.20,183.78,62697158.0,998068972.0,3.225590e+09,0.019437
...,...,...,...,...,...,...,...,...,...
603706,2026-02-05,27.94,27.99,27.48,27.48,975279.0,20545545.0,1.893827e+08,0.005150
600603,2026-02-05,1222.35,1237.17,1207.53,1226.05,9066800.0,59791995.0,1.193329e+09,0.007598
002818,2026-02-05,32.24,32.68,32.07,32.48,3320320.0,39179662.0,2.983572e+08,0.011129


In [23]:
pd.concat([daily_stock, daily_factor], axis=1, ignore_index=False)

,date,open,high,low,close,volume,amount,outstanding_share,turnover,BETA10,...,VSUMP10,VSUMP20,VSUMP30,VSUMP5,VSUMP60,WVMA10,WVMA20,WVMA30,WVMA5,WVMA60
601599,2026-02-05,14.61,16.02,14.61,16.02,46393702.0,222835685.0,1.160542e+09,0.039976,0.017894,...,0.531175,0.508066,0.523043,0.063069,0.519381,0.872750,1.301405,1.523789,0.359468,1.919617
600985,2026-02-05,64.26,64.26,62.25,63.30,38946182.0,489317462.0,2.693259e+09,0.014461,0.005448,...,0.447720,0.503160,0.531321,0.496022,0.515101,1.279526,1.513613,1.687654,1.090034,1.936573
600609,2026-02-05,131.88,132.43,130.49,130.77,12058341.0,57167454.0,1.304559e+09,0.009243,-0.006483,...,0.416180,0.495337,0.493712,0.204202,0.489092,0.931434,1.484573,1.768540,0.408766,1.634550
002700,2026-02-05,45.85,45.97,45.23,45.36,6959400.0,51338401.0,2.634649e+08,0.026415,0.002776,...,0.427196,0.530546,0.526102,0.491445,0.504189,2.000138,2.005364,1.964230,1.645865,1.699142
000066,2026-02-05,184.24,185.98,183.20,183.78,62697158.0,998068972.0,3.225590e+09,0.019437,-0.014029,...,0.062839,0.495922,0.508224,0.236545,0.504052,1.312127,1.581756,1.942356,0.832797,2.670481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603706,2026-02-05,27.94,27.99,27.48,27.48,975279.0,20545545.0,1.893827e+08,0.005150,-0.000249,...,0.427725,0.486352,0.492128,0.147730,0.490129,0.791111,1.004792,1.072203,0.964047,1.922273
600603,2026-02-05,1222.35,1237.17,1207.53,1226.05,9066800.0,59791995.0,1.193329e+09,0.007598,-0.004202,...,0.180866,0.489384,0.514574,0.463246,0.474661,1.020973,1.835047,1.544694,0.712473,1.416745
002818,2026-02-05,32.24,32.68,32.07,32.48,3320320.0,39179662.0,2.983572e+08,0.011129,-0.001179,...,0.502644,0.516205,0.506437,0.453300,0.507040,1.843580,2.331925,2.654912,0.701273,2.526033
002607,2026-02-05,23.59,23.76,23.35,23.35,122417090.0,348756320.0,5.083997e+09,0.024079,-0.006468,...,0.408684,0.434290,0.492708,0.158718,0.502061,0.787948,1.283147,1.377216,0.878799,1.505910


In [24]:
data_handler.get_stock_factors(stock_code="601599", start_date=date(2026, 2, 5), end_date=date(2026, 2, 5))

factor_name,BETA10,BETA20,BETA30,BETA5,BETA60,CNTD10,CNTD20,CNTD30,CNTD5,CNTD60,...,VSUMP10,VSUMP20,VSUMP30,VSUMP5,VSUMP60,WVMA10,WVMA20,WVMA30,WVMA5,WVMA60
trade_date,,,,,,,,,,,,,,,,,,,,,
2026-02-05,0.017894,0.007252,0.004907,0.013358,0.002041,0.6,0.25,0.166667,0.6,0.05,...,0.531175,0.508066,0.523043,0.063069,0.519381,0.87275,1.301405,1.523789,0.359468,1.919617


In [25]:
data_handler.close()